In [1]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus 


8

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    1811854, 3772146, 6407100, 6926227, 1340378, 
    68679, 3822303, 4058970, 1264160, 1100338, 
    6552126, 725363, 8362134, 9446122, 5256199, 
    4686339, 3830906, 7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [20]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2565 (20.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2625 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2596 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2666 (21.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2652 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2570 (20.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2646 (20.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2166 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2150 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2190 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2194 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2165 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2145 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2198 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [9]:
import os

output_folder_path = "result_experiments_xg/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 1000  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative, runHyperparameterSearch_Content
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
import os, multiprocessing
from functools import partial
from skopt.space import Real, Integer, Categorical


2024-01-08 10:02:00.778478: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-08 10:02:00.778516: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
from concurrent.futures import ProcessPoolExecutor
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
import time
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender

In [12]:
import optuna

host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "optuna"
password = "your_password"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

study_name = "P3Alpha_8FOLD"

study = optuna.create_study(study_name=study_name, storage=storage, load_if_exists=True, direction="maximize", sampler=optuna.samplers.CmaEsSampler())

model = P3alphaRecommender

[I 2024-01-08 10:02:02,333] Using an existing study with name 'P3Alpha_8FOLD' instead of creating a new one.


In [13]:
def run_fold(fold, **hyperparams):
    recommender = model(urm_train_validation[fold])
    recommender.fit(**hyperparams)

    metric_result = evaluator_test[fold].evaluateRecommender(recommender)[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, **hyperparams}
    print(res)
    return res

def run_fold_wrapper(fold, hyperparams):
    return run_fold(fold, **hyperparams)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


def objective(trial):

    params = {
        "topK": trial.suggest_int("topK",10, 1000),
        "alpha": trial.suggest_float("alpha",low = 0, high = 2),
    }

    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
    print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

In [14]:
study.enqueue_trial({'topK':50, 'alpha':0.7745100325869})
study.optimize(objective, n_trials=200)

P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 201 ( 1.6%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.


P3alphaRecommender: URM Detected 122 ( 0.5%) items with no interactions.

P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 220 ( 1.7%) users with no interactions.

P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 106 ( 0.5%) items with no interactions.


P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 114 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 118 ( 0.5%) items wit

[I 2024-01-08 10:02:15,261] Trial 2 finished with value: 0.04207924823171881 and parameters: {'topK': 50, 'alpha': 0.7745100325869}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.042755458243904115, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 1, 'MAP': 0.04228147361156576, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 2, 'MAP': 0.04210906545492158, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 3, 'MAP': 0.042676105518168604, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 4, 'MAP': 0.04208064501461856, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 5, 'MAP': 0.04152745821477405, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 6, 'MAP': 0.04183271764168922, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 7, 'MAP': 0.041371062154108615, 'topK': 50, 'alpha': 0.7745100325869}]
P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 201 ( 1.6%) users with no interactions.P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3al

[I 2024-01-08 10:02:41,619] Trial 3 finished with value: 0.022088496839890275 and parameters: {'topK': 420, 'alpha': 1.095128944847853}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.021974721706864524, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 1, 'MAP': 0.021985061536691948, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 2, 'MAP': 0.02219263503002018, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 3, 'MAP': 0.02229295471539819, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 4, 'MAP': 0.022967892494532398, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 5, 'MAP': 0.021575385101783653, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 6, 'MAP': 0.022427178539961578, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 7, 'MAP': 0.02129214559386973, 'topK': 420, 'alpha': 1.095128944847853}]
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.
P3alphaRecommender: URM Detected 239 ( 1.9%) use

...

[{'Fold': 0, 'MAP': 0.043248185637892074, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 1, 'MAP': 0.04308506395820524, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 2, 'MAP': 0.04311666317241604, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 3, 'MAP': 0.04302176232423333, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 4, 'MAP': 0.04318551558883893, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 5, 'MAP': 0.042344892815921756, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 6, 'MAP': 0.04265454845993741, 'topK': 62, 'alpha': 0.7545584560532148}, {'Fold': 7, 'MAP': 0.04206844857994337, 'topK': 62, 'alpha': 0.7545584560532148}]
P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users wi

[I 2024-01-08 11:00:02,457] Trial 200 finished with value: 0.042816707196731196 and parameters: {'topK': 58, 'alpha': 0.7525489916250321}. Best is trial 185 with value: 0.042847568800941496.


[{'Fold': 0, 'MAP': 0.04321584694483902, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 1, 'MAP': 0.04305166600075128, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 2, 'MAP': 0.043104771324048274, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 3, 'MAP': 0.043005895387646355, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 4, 'MAP': 0.04311873009810613, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 5, 'MAP': 0.04236697480860314, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 6, 'MAP': 0.042668663115066716, 'topK': 58, 'alpha': 0.7525489916250321}, {'Fold': 7, 'MAP': 0.04200110989478862, 'topK': 58, 'alpha': 0.7525489916250321}]
P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users 

[I 2024-01-08 11:00:16,179] Trial 201 finished with value: 0.04279486382530455 and parameters: {'topK': 64, 'alpha': 0.7551010593384083}. Best is trial 185 with value: 0.042847568800941496.


[{'Fold': 0, 'MAP': 0.04306380885687667, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 1, 'MAP': 0.043145325572385596, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 2, 'MAP': 0.04316153011740737, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 3, 'MAP': 0.04288117123525073, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 4, 'MAP': 0.04310370279433637, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 5, 'MAP': 0.04231709272546079, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 6, 'MAP': 0.04259575445731991, 'topK': 64, 'alpha': 0.7551010593384083}, {'Fold': 7, 'MAP': 0.04209052484339901, 'topK': 64, 'alpha': 0.7551010593384083}]


In [15]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
import optuna

host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "optuna"
password = "your_password"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

study_name = "RP3_8FOLD"

study = optuna.create_study(study_name=study_name, storage=storage, load_if_exists=True, direction="maximize", sampler=optuna.samplers.CmaEsSampler())

model = RP3betaRecommender

[I 2024-01-08 11:00:16,245] Using an existing study with name 'RP3_8FOLD' instead of creating a new one.


In [16]:
def run_fold(fold, **hyperparams):
    recommender = model(urm_train_validation[fold])
    recommender.fit(**hyperparams)

    metric_result = evaluator_test[fold].evaluateRecommender(recommender)[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, **hyperparams}
    print(res)
    return res

def run_fold_wrapper(fold, hyperparams):
    return run_fold(fold, **hyperparams)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


def objective(trial):

    params = {
        "topK": trial.suggest_int("topK",1, 1000),
        "alpha": trial.suggest_float("alpha",low = 0, high = 2),
        "beta": trial.suggest_float("beta",low = 0, high = 2),
    }

    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
    print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

In [18]:
# param_id, trial_id, param_name, param_value, distribution_json, value
#study.enqueue_trial({'alpha': 0.2765812198526 , 'beta': 0.1793692584175053, 'topK': 23})
#study.enqueue_trial({'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933})

study.optimize(objective, n_trials=75)

RP3betaRecommender: URM Detected 201 ( 1.6%) users with no interactions.RP3betaRecommender: URM Detected 223 ( 1.8%) users with no interactions.RP3betaRecommender: URM Detected 229 ( 1.8%) users with no interactions.RP3betaRecommender: URM Detected 217 ( 1.7%) users with no interactions.RP3betaRecommender: URM Detected 220 ( 1.7%) users with no interactions.RP3betaRecommender: URM Detected 227 ( 1.8%) users with no interactions.RP3betaRecommender: URM Detected 243 ( 1.9%) users with no interactions.RP3betaRecommender: URM Detected 239 ( 1.9%) users with no interactions.







RP3betaRecommender: URM Detected 122 ( 0.5%) items with no interactions.RP3betaRecommender: URM Detected 118 ( 0.5%) items with no interactions.RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.RP3betaRecommender: URM Detected 126 ( 0.6%) items with no interactions.

RP3betaRecommender: URM Detected 106 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 113 ( 0.5%) items wi

[I 2024-01-08 12:10:25,147] Trial 201 finished with value: 0.048527116189757774 and parameters: {'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}. Best is trial 1 with value: 0.04869067440247042.


[{'Fold': 0, 'MAP': 0.048482624110250824, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 1, 'MAP': 0.04862505372731401, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 2, 'MAP': 0.04852557032256553, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 3, 'MAP': 0.04823556321545674, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 4, 'MAP': 0.04893059856107751, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 5, 'MAP': 0.04845892348489384, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 6, 'MAP': 0.04863590558990455, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}, {'Fold': 7, 'MAP': 0.04832269050659918, 'topK': 28, 'alpha': 0.22539119554958706, 'beta': 0.13717922664361337}]
RP3betaRecommender: URM Detected 229 ( 1.8%) users with no interactions.RP3betaRecommender: URM Detect

[I 2024-01-08 12:10:40,063] Trial 202 finished with value: 0.04799381995895243 and parameters: {'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}. Best is trial 1 with value: 0.04869067440247042.


[{'Fold': 0, 'MAP': 0.048082824038707034, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 1, 'MAP': 0.04825960205465426, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 2, 'MAP': 0.04773673551860815, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 3, 'MAP': 0.04764940225419985, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 4, 'MAP': 0.048792844487488524, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 5, 'MAP': 0.047912485874937794, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 6, 'MAP': 0.04802251899772729, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}, {'Fold': 7, 'MAP': 0.04749414644529648, 'topK': 49, 'alpha': 0.29318552699090206, 'beta': 0.18328645251801864}]
RP3betaRecommender: URM Detected 223 ( 1.8%) users with no interactions.RP3betaRecommender: URM Dete

...

[{'Fold': 0, 'MAP': 0.048519657863145986, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 1, 'MAP': 0.048890239638953184, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 2, 'MAP': 0.048393111615499154, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 3, 'MAP': 0.04854710002857345, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 4, 'MAP': 0.04946402237651227, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 5, 'MAP': 0.04829228934179909, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 6, 'MAP': 0.04852468680857966, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}, {'Fold': 7, 'MAP': 0.04844721538040563, 'topK': 30, 'alpha': 0.2969238227447483, 'beta': 0.16956687180669425}]
RP3betaRecommender: URM Detected 243 ( 1.9%) users with no interactions.RP3betaRecommender: URM Detected 227

[I 2024-01-08 12:24:28,423] Trial 274 finished with value: 0.04844850302088899 and parameters: {'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}. Best is trial 1 with value: 0.04869067440247042.


[{'Fold': 0, 'MAP': 0.04850561815635416, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 1, 'MAP': 0.048885245268970125, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 2, 'MAP': 0.04813410404360875, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 3, 'MAP': 0.048324886925280204, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 4, 'MAP': 0.04937551436119242, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 5, 'MAP': 0.04813413023648469, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 6, 'MAP': 0.048440283218157194, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}, {'Fold': 7, 'MAP': 0.04778824195706439, 'topK': 36, 'alpha': 0.3264783369760893, 'beta': 0.16710892973634267}]
RP3betaRecommender: URM Detected 229 ( 1.8%) users with no interactions.RP3betaRecommender: URM Detected 217

[I 2024-01-08 12:24:41,125] Trial 275 finished with value: 0.04863244940199216 and parameters: {'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}. Best is trial 1 with value: 0.04869067440247042.


[{'Fold': 0, 'MAP': 0.048421251076188825, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 1, 'MAP': 0.04899167453658369, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 2, 'MAP': 0.04840745323780402, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 3, 'MAP': 0.048591881569429306, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 4, 'MAP': 0.04934810449849193, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 5, 'MAP': 0.04836035815260119, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 6, 'MAP': 0.04855379946953526, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}, {'Fold': 7, 'MAP': 0.04838507267530307, 'topK': 31, 'alpha': 0.3332259357345812, 'beta': 0.16230609429719137}]


In [24]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import optuna

host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "optuna"
password = "your_password"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

study_name = "ITEMKNN_8FOLD"

study = optuna.create_study(study_name=study_name, storage=storage, load_if_exists=True, direction="maximize", sampler=optuna.samplers.CmaEsSampler())

model = ItemKNNCFRecommender

[I 2024-01-08 14:28:49,828] Using an existing study with name 'ITEMKNN_8FOLD' instead of creating a new one.


In [25]:
def run_fold(fold, **hyperparams):
    recommender = model(urm_train[fold])
    recommender.fit(**hyperparams)

    metric_result = evaluator_validation[fold].evaluateRecommender(recommender)[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, **hyperparams}
    print(res)
    return res

def run_fold_wrapper(fold, hyperparams):
    return run_fold(fold, **hyperparams)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


def objective(trial):

    params = {
        "topK": trial.suggest_int("topK",1, 1000),
        "shrink": trial.suggest_float("shrink",low = 0, high = 100),
    }

    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
    print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

In [26]:
study.enqueue_trial({'topK':15 , 'shrink': 8.782558252996342})

study.optimize(objective, n_trials=500)

ItemKNNCFRecommender: URM Detected 443 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 467 ( 3.7%) users with no interactions.ItemKNNCFRecommender: URM Detected 474 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Detected 453 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 481 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Detected 450 ( 3.6%) users with no interactions.

ItemKNNCFRecommender: URM Detected 323 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 446 ( 3.5%) users with no interactions.ItemKNNCFRecommender: URM Detected 328 ( 1.5%) items with no interactions.

ItemKNNCFRecommender: URM Detected 355 ( 1.6%) items with no interactions.

ItemKNNCFRecommender: URM Detected 315 ( 1.4%) items with no interactions.
ItemKNNCFRecommender: URM Detected 481 ( 3.8%) users with no interactions.

ItemKNNCFRecommender: URM 

[I 2024-01-08 14:29:02,164] Trial 179 finished with value: 0.027600376417323935 and parameters: {'topK': 15, 'shrink': 8.782558252996342}. Best is trial 172 with value: 0.027639628553478953.


[{'Fold': 0, 'MAP': 0.02792421277058432, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 1, 'MAP': 0.02740203608325057, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 2, 'MAP': 0.027238973296282668, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 3, 'MAP': 0.028003879115491827, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 4, 'MAP': 0.027123810157652757, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 5, 'MAP': 0.02786469311003669, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 6, 'MAP': 0.027799164096840044, 'topK': 15, 'shrink': 8.782558252996342}, {'Fold': 7, 'MAP': 0.02744624270845262, 'topK': 15, 'shrink': 8.782558252996342}]
ItemKNNCFRecommender: URM Detected 453 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 446 ( 3.5%) users with no interactions.ItemKNNCFRecommender: URM Detected 481 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Detected 450 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 467 ( 3.

[I 2024-01-08 14:29:10,407] Trial 180 finished with value: 0.027654249371273875 and parameters: {'topK': 15, 'shrink': 11.718523336217158}. Best is trial 180 with value: 0.027654249371273875.


[{'Fold': 0, 'MAP': 0.02793454606767429, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 1, 'MAP': 0.027502080628516334, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 2, 'MAP': 0.027454761430563163, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 3, 'MAP': 0.028068086674434615, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 4, 'MAP': 0.027182032608058522, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 5, 'MAP': 0.027923700323309916, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 6, 'MAP': 0.027783335225230445, 'topK': 15, 'shrink': 11.718523336217158}, {'Fold': 7, 'MAP': 0.027385452012403715, 'topK': 15, 'shrink': 11.718523336217158}]
ItemKNNCFRecommender: URM Detected 453 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 450 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 481 ( 3.8%) users with no interactions.

ItemKNNCFRecommender: URM Detected 474 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Dete

...

[{'Fold': 0, 'MAP': 0.027924445200827652, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 1, 'MAP': 0.027494816262668146, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 2, 'MAP': 0.02755968187580425, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 3, 'MAP': 0.02803286901737574, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 4, 'MAP': 0.02720996924276365, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 5, 'MAP': 0.027928782835652588, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 6, 'MAP': 0.027915192563583636, 'topK': 15, 'shrink': 12.186608057982475}, {'Fold': 7, 'MAP': 0.027410968457305684, 'topK': 15, 'shrink': 12.186608057982475}]
ItemKNNCFRecommender: URM Detected 481 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Detected 446 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 450 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 474 ( 3.8%) users with no interactions.

ItemKNNCFRecommender: URM Detec

[I 2024-01-08 15:40:14,466] Trial 488 finished with value: 0.027684590681997665 and parameters: {'topK': 15, 'shrink': 12.187409293702336}. Best is trial 193 with value: 0.027684590681997665.


[{'Fold': 0, 'MAP': 0.027924445200827652, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 1, 'MAP': 0.027494816262668146, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 2, 'MAP': 0.02755968187580425, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 3, 'MAP': 0.02803286901737574, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 4, 'MAP': 0.02720996924276365, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 5, 'MAP': 0.027928782835652588, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 6, 'MAP': 0.027915192563583636, 'topK': 15, 'shrink': 12.187409293702336}, {'Fold': 7, 'MAP': 0.027410968457305684, 'topK': 15, 'shrink': 12.187409293702336}]
ItemKNNCFRecommender: URM Detected 450 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 453 ( 3.6%) users with no interactions.ItemKNNCFRecommender: URM Detected 474 ( 3.8%) users with no interactions.ItemKNNCFRecommender: URM Detected 467 ( 3.7%) users with no interactions.ItemKNNCFRecommender: URM Detected

[W 2024-01-08 15:40:34,882] Trial 489 failed with parameters: {'topK': 15, 'shrink': 12.186499539076861} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_35993/4050470509.py", line 26, in objective
    fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/concurrent/futures/process.py", line 484, in _chain_from_iterable_of_lists
    for element in iterable:
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/concurrent/futures/_base.py", line 619, in result_iterator
    yield fs.pop().result()
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/concurrent/futures/_base.py", line 439, in result
    self._condition.wait(timeout)
  File "/

KeyboardInterrupt: 